#1. 협업 필터링

협업 필터링 알고리즘은 큰 무리의 사람들을 검색해서 여러분과 유사한 취향을 가진 작은 집합을 발견하는 방식으로 동작한다. <br>
그것은 사람들이 좋아할 만한 여러 것들을 보고 결합하여 추천 목록을 생성한다. <br>
어떤 사람들이 유사한지 결정하거나 그들의 선택을 결합하여 목록을 만드는 여러가지 방법이 있는데 이 장에서는 몇몇 가지 방법만 소개 

#2. 선호정보 수집

선호 정보를 수집 위와 같이 자료형에 데이터를 추가 한다. 

In [1]:
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 
 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 
 'You, Me and Dupree': 3.5}, 
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0, 
 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0}, 
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

critics라는 딕션어리형 자료형에 영화에 대한 정보를 입력한다.<br>
이와 같은 정보는 목적에 따라 다른 선호도 값을 가진다. <br>
공연 티켓의 경우 구매 : 1, 비 구매 : 0 <br>
온라인 쇼핑몰의 경우 구매 : 2 , 찾음 : 1 , 비구매 <br>
사이트 추천의 경우 좋음 : 1 , 모름 : 0 , 싫음 -1<br>
과 같은 기준으로 선호도를 표시 한다. 

<img src=./img/prefer_table.png>

critics 딕션어리에 값이 제대로 있는지 확인

In [2]:
critics['Lisa Rose']['Lady in the Water']

2.5

In [3]:
critics['Toby']['Snakes on a Plane']=4.5
critics['Toby']

{'Snakes on a Plane': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 1.0}

#3. 유사 사용자 찾기

사람들이 선호하는 정보를 수집 했다면<br>
다음 단계는 유사한 사용자를 찾는것이다.<br>
유사한 사용자를 찾는 방법은 <br>
유클리디안 거리 점수와 피어슨 상관 점수가 있다. 

## 유클리디안 거리 점수

<img src=./img/eucdlian-distance.png>

사람들 간의 선호 공간

snakes, dupree라는 두개의 축을 기준으로 사람을 배열(선호공간)

Tobby는 Snakeds영화 축에서 4.5점<br>
Dupree 영화축에서 1.0에 위치

선호 공간에서 두 사람간의 위치가 가까울 수록 그들 간의 선호도가 더 유사함을 의미.<br>
이 차트는 2차원 이기 때문에 동시에 두개의 순위만 볼 수 있다.<br> 
위의 예제는 2차원 데이타이기 때문에 2차원 데이터로만 표시, 실 데이타는 다차원 데이터로 표시

###유클리디안 거리 점수

<img src=./img/euclidean-distance-score.png>

python code

In [4]:
from math import sqrt
sqrt(pow(5-4,2) + pow(2-1,2))

1.4142135623730951

위 공식으로 거리 계산하면 유사한 사람일수록 작은 값을 가진다.<br>
하지만 사람 간에 유사할수록 더 높은 값을 주는 함수가 더 좋다. <br>
그렇게 하기 위해서는 다음과 같이 함수에 1을 더 하고 역을 위하면 더 좋은 점수를 구할 수 있다.

<img src=./img/ext-euc.png>

좀 더 효율적인 유클리디안 점수 

In [5]:
from math import sqrt
1/(1+sqrt(pow(5-4,2) + pow(2-1,2)))

0.4142135623730951

In [6]:
1/ (1 + 1.4142)

0.4142158893215144

이 함수의 장점은 두 사용자간의 거리가 0~1로 표시 가능<br>
가장 멀리 떨어진 두 사용자간의 거리 : 0 <br>
가장 가까운 두 사용자의 거리 : 1

In [7]:
from math import sqrt

# Returns a distance-based similarity score for person1 and person2
def sim_distance(prefs,person1,person2):
  # 공통항목 추출
  si={}
  for item in prefs[person1]: 
    if item in prefs[person2]: si[item]=1

  # 공통 평가 항목이 없을 경우 0
  if len(si)==0: return 0

  # 모든 차이의 값을 제곱하여 더함
  sum_of_squares=sum([pow(prefs[person1][item]-prefs[person2][item],2) 
                      for item in prefs[person1] if item in prefs[person2]])

  return 1/(1+sum_of_squares)

이제 실제 데이타를 적용

In [8]:
sim_distance(critics, 'Lisa Rose', 'Gene Seymour')

0.14814814814814814

위 결과는 "Lisa Rose"와 "Gene Seymour" 두 사람간의 유사도 

###피어슨 상관점수

상관분석(Correlation Analysis)은 확률론과 통계학에서 두 변수간에 어떤 선형적 관계를 갖고 있는 지를 분석하는 방법이다. 두변수는 서로 독립적인 관계로부터 서로 상관된 관계일 수 있으며 이때 두 변수간의 관계의 강도를 상관관계(Correlation, Correlation coefficient)라 한다. 상관분석에서는 상관관계의 정도를 나타내는 단위로 모상관계수 ρ를 사용한다.

이 방법을 시각화하기 위해 차트에 두 명의 평론가의 평가 결과를 출력

<img src=./img/p-score.png width=600>

산포도로 두 영화 평론가를 비교하기

이 차트에서 한개의 직선이 표시되어 있다. <br>
이 직선을 최적 맞춤선(best-fit line)이라고 부르는데, 이는 차트 내 모든 항목글과 가능한 가장 가까운 직선임을 의미<br>
모든 평론가가 모든 경화를 동일하게 평가했다면 이 선은 대각선이 될것이고 이 선 위에 모든 항목 들이 있게 된다. <br>
이 경우 완전 상관점수인 1 값이된다. <br>
위 도표에서 몇몇 영화의 경우 평가점수가 달라 상관점수가 0.4 쯤 된다.

<img src=./img/p-score1.png width=600>

피어슨 점수는 점수 부풀리시(grade inflation)란 흥미로운 특성이 있다. <br>
그림에서 잭 매튜가 리사 로스보다 후한 점수를 주어다.<br>
그래도 두 사람이 상대적으로 유사한 선호도를 가졌기 때문에 한 직선으로 잘 표현되어 있다. 

피어슨 상관관계 점수 계산 python code

In [14]:
def sim_pearson(prefs,p1,p2):
  # Get the list of mutually rated items
  si={}
  for item in prefs[p1]: 
    if item in prefs[p2]: si[item]=1

  # if they are no ratings in common, return 0
  if len(si)==0: return 0

  # Sum calculations
  n=len(si)
  
  # Sums of all the preferences
  sum1=sum([prefs[p1][it] for it in si])
  sum2=sum([prefs[p2][it] for it in si])
  
  # Sums of the squares
  sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
  sum2Sq=sum([pow(prefs[p2][it],2) for it in si])	
  
  # Sum of the products
  pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
  
  # Calculate r (Pearson score)
  num=pSum-(sum1*sum2/n)
  den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
  if den==0: return 0

  r=num/den

  return r

이 함수는 -1~1 사이의 값을 리턴한다. 1은 두 사람의 상관 계수가 모드낭목에서 같은 점수를 주어졌을때 점수이다.<br>
다음과 같이 상관 관계 점수를 계산하면

In [16]:
print sim_pearson(critics, 'Lisa Rose', 'Gene Seymour')

0.396059017191


###유사도 측정 지표 선택 방법

유사도 측정 지표 선택 방법은 유클리디안거리 계산법, 피어슨 상관계수 외 다른 여러 가지 방법이 있다.<br>
자카드 상관계수[http://socurites.com/144]<br>
맨하튼 거리 [http://www.nicklib.com/algorithm/1697]<br>

###평론가 순위

두 사람을 비교하는 함수를 만들었으므로, 주어진 사람을 기준으로 다른 모든 사람들과의 점수를 계산하여 가장 근접한 상대편을 찾는 함수를 만들수 있다. <br>
이 경우 나와 유사한 성향을 가진 영화 평론가를 찾아 영화를 고를때 어느 평론가의 의견을 믿을지 판단 하면 재미 있을 것이다. 

In [22]:
#선호도 딕셔너리에서 최적의 상대편을 구함
#결과 개수와 유사도 함수는 옵션 사항
def topMatches(prefs, person, n=5, similarity=sim_pearson):
    scores = [(similarity(prefs, person, other), other)
             for other in prefs if other != person ]
    
    #최고점이 상단에 오도록 목적을 정렬
    scores.sort()
    scores.reverse()
    return scores[0:n]

위의 코드를 이용하여 특정 이용자와 가장 가까운 평론가를 찾아 보자

In [23]:
topMatches(critics, 'Toby', n=3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

'Toby'와 가장 유사한 평론가가 검출된 결과를 확인 할 수 있다. <br>
 similarity인자를 바꾸면 다른 방법으로 유사한 사용자를 추천 받을 수 있다.

#4. 항목 추천

읽어 볼만한 가치 있는 평론가를 찾는 것도 중요하지만, 정말 필요한건 영화 추천이다. <br>
나와 가장 유사한 취향을 가진 사람을 찾아 그가좋아하지만 내가 안 본 영화를 찾을 수도 있다. <br>
그러나 이건 너무 관대한 판단이다.  <br>
평론가 순위를 결정하는 가중점수를 만들어 항목점수를 만들어 항목점수를 계산해야 한다. <br>
모든 다른 평론가들의 평가를 계산한후 나와의 유사도를 그들이 각 영화에 준 점수와 곱한다.<br>
다음 표와 같이 수행

<img src=./img/table2-2.png width=600>

위 표는 각 평론가와의 상관점수와 내가 평가하지 않은 세 영화(bight, lady, luck)에 대한 평론가들의 평가 점수를 표시한다.<br>
S.x로 시작하는 세로줄은 유사도를 평가점수로 곱한 값<br>
나와 유사한 평론가들의 평가가 나와는 다른 평론가들의 평가보다 전체 점수에 더 큰 기여<br>


많은 평론가가 평가한 영화가 이득을 얻을수 있다. <br>
이를 보정하기 위해 평론의 총합에서 평론한 사람의 수를 나누어서 점수를 보정 -> 마지막 가로줄<br>


In [29]:
# Gets recommendations for a person by using a weighted average
# of every other user's rankings
# 다른 사람과의 순위의 가중평균값을 이용해서 특정 사람에 추천
def getRecommendations(prefs,person,similarity=sim_pearson):
  totals={}
  simSums={}
  for other in prefs:
    # don't compare me to myself
    #자신과는 비교 하지 않는다. 
    if other==person: continue
    sim=similarity(prefs,person,other)

    # ignore scores of zero or lower
    # 0또는 0보다 작은 값은 비교하지 않는다.
    if sim<=0: continue
    for item in prefs[other]:
      # only score movies I haven't seen yet
      # 내가 보지 못한 영화만을 대상으로 한다.
      if item not in prefs[person] or prefs[person][item]==0:
        # Similarity * Score
        # 유사도와 점수를 곱
        totals.setdefault(item,0)
        totals[item]+=prefs[other][item]*sim
        # Sum of similarities
        # 유사도 합
        simSums.setdefault(item,0)
        simSums[item]+=sim

  # Create the normalized list
  # 결과를 정규화(노말라이즈)
  rankings=[(total/simSums[item],item) for item,total in totals.items()]

  # Return the sorted list
  rankings.sort()
  rankings.reverse()
  return rankings

critics 자료를 통해 Toby에게 추천할 영화(피어슨 상관계수를 이용할때)

In [30]:
getRecommendations(critics, 'Toby')

[(3.3477895267131013, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.5309807037655645, 'Just My Luck')]

critics 자료를 통해 Toby에게 추철한 영화(유클리디안 유사도 평가 기준)

In [31]:
getRecommendations(critics, 'Toby', similarity=sim_distance)

[(3.5002478401415877, 'The Night Listener'),
 (2.7561242939959363, 'Lady in the Water'),
 (2.461988486074374, 'Just My Luck')]

유사도를 평가하는 다른 방법을 이용하면 같은 데이터에도 다른 결과를 얻을 수 있다.

#5. 항목 매칭

한 제품과 다른 제품이 얼마나 유사한지를 찾는 방법을 기술

<img src=./img/figure2-4.png width=600>

위 그림은 아마존에서 "파이선 프로그래밍"과 유사한 제품을 표시

데이터의 변환 필요

<img src=./img/human-base.png>

<img src=./img/item-base.png>

평론가를 기준으로 구성된 딕션어리를 영화를 기준으로 구성된 딕션어리로 변환<br>
python 코드를 통해 간단히 구현

In [33]:
def transformPrefs(prefs):
  result={}
  for person in prefs:
    for item in prefs[person]:
      result.setdefault(item,{})
      
      # Flip item and person
      # 영화와 평론가의 위치를 바꿈
      result[item][person]=prefs[person][item]
  return result

슈퍼맨 리턴즈와 유사한 영화를 조회한 결과

In [34]:
movies = transformPrefs(critics)
topMatches(movies, 'Superman Returns')

[(0.6579516949597695, 'You, Me and Dupree'),
 (0.4879500364742689, 'Lady in the Water'),
 (0.11180339887498941, 'Snakes on a Plane'),
 (-0.1798471947990544, 'The Night Listener'),
 (-0.42289003161103106, 'Just My Luck')]

음의 상관 관계는 좋아하지 않는다라고 해석가능

<img src=./img/figure2-5.png width=600>

특정 영화를 추천하는 평론가를 추출할 수 도 있다.

In [35]:
getRecommendations(movies, 'Just My Luck')

[(4.0, 'Michael Phillips'), (3.0, 'Jack Matthews')]

#6. 딜리셔스 링크 추천 기능 만들기

온라인 북마크 사이트인 딜리셔스의 데이터를 이용하여 추천 기능을 만든다.

설치해야할 라이브러리 : <br>
pydelicious : !easy_install pydelicious<br>
feedparser : !pip install feedparser

In [38]:
!pip install feedparser
!pip install --upgrade feedparser 

/usr/local/lib/python2.7/site-packages/pip/_vendor/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
    100% |████████████████████████████████| 192kB 649kB/s 
  Found existing installation: feedparser 5.2.0
    Uninstalling feedparser-5.2.0:
      Successfully uninstalled feedparser-5.2.0
  Running setup.py install for feedparser


<img src=./img/figure2-6.png width=600>

In [53]:
from pydelicious import DeliciousAPI
pydelicious.get_popular(tag='programming')

[{'count': '',
  'description': u'something went wrong',
  'dt': '',
  'extended': '',
  'hash': '',
  'href': '',
  'tags': '',
  'user': ''}]

API오류 생략 [http://stackoverflow.com/questions/29543799/pydelicious-get-popularprogramming-doesnt-return-any-valid-url]

#7. 항목 기반 필터링

##사용자 기반의 협력 필터링 : <BR>
수 많은 사람들의 평가를 기반으로 처리.<BR>
많은 사용자가 있을때 유리.<BR>
사용자를 그룹화 하는 작업에 오랜 시간 소요 <BR>
수많은 제품으로 인해 사용자간의 구입이력이 겹치지 않을때 불리<BR>

##항목 기반 협력 필터링 :<BR>
항목 별로 유사한 항목을 미리 계산<BR>
사용자에게 추천 할때 사용자가 평가한 상위 항목을 보고 이것과 가장 유사한 항목들의 가중치 목록 생성<br>
장점 : 항목은 사람에 비해 자주 바뀌지 않는다.

##항목 비교 데이터 세트 생성

항목간의 비교를 위해 먼저 유사 항목들을 담은 완전한 데이터 세트를 생성

유사 항목을 구하는 알고리즘

In [54]:
def calculateSimilarItems(prefs,n=10):
  # Create a dictionary of items showing which other items they
  # are most similar to.
  result={}
  # Invert the preference matrix to be item-centric
  itemPrefs=transformPrefs(prefs)
  c=0
  for item in itemPrefs:
    # Status updates for large datasets
    c+=1
    if c%100==0: print "%d / %d" % (c,len(itemPrefs))
    # Find the most similar items to this one
    scores=topMatches(itemPrefs,item,n=n,similarity=sim_distance)
    result[item]=scores
  return result

유사도 데이터셋 확인

In [55]:
itemsim = calculateSimilarItems(critics)
itemsim

{'Just My Luck': [(0.2222222222222222, 'Lady in the Water'),
  (0.18181818181818182, 'You, Me and Dupree'),
  (0.15384615384615385, 'The Night Listener'),
  (0.10526315789473684, 'Snakes on a Plane'),
  (0.06451612903225806, 'Superman Returns')],
 'Lady in the Water': [(0.4, 'You, Me and Dupree'),
  (0.2857142857142857, 'The Night Listener'),
  (0.2222222222222222, 'Snakes on a Plane'),
  (0.2222222222222222, 'Just My Luck'),
  (0.09090909090909091, 'Superman Returns')],
 'Snakes on a Plane': [(0.2222222222222222, 'Lady in the Water'),
  (0.18181818181818182, 'The Night Listener'),
  (0.16666666666666666, 'Superman Returns'),
  (0.10526315789473684, 'Just My Luck'),
  (0.05128205128205128, 'You, Me and Dupree')],
 'Superman Returns': [(0.16666666666666666, 'Snakes on a Plane'),
  (0.10256410256410256, 'The Night Listener'),
  (0.09090909090909091, 'Lady in the Water'),
  (0.06451612903225806, 'Just My Luck'),
  (0.05333333333333334, 'You, Me and Dupree')],
 'The Night Listener': [(0.28

항목 유사도를 최신으로 유지하기 충분할 정도로만 수행.<br>
사용자와 평가 수가 작을 때 좀 더 자주 돌려야 함 : 안정화가 덜 되었기 때문 <br>
사용자와 평가 수가 점점 많아질수록 항목들간의 유사도 점수는 안정화 되어 간다.

##추천 생성

<img src=./img/table2-3.png width=600>

항목 기반 방식을 사용하여 추천 항목을 발견기 위한 테이블<br>
이전의 표와는 달리 평론가가 전혀 포함되어 있지 않고 내가 평가한 영화와 내가 평가하지 않은 영화간의 비교만 있음

Superman의 평가가 4점이고 Night와 Superman의 유사도가 0.103이기 때문에 Night에 대한 평가(R.xNight)는 0.412가 된다. <br>
Normalized항목은 Night에 대한 기대값(1.378)에 Night의 유사도 합계(0.433)을 나눈다.<br> 
Normalized Night(3.183) = 1.378/ 0.433

다음 코드에서 위와 같은 역할을 수행

In [56]:
def getRecommendedItems(prefs,itemMatch,user):
  userRatings=prefs[user]
  scores={}
  totalSim={}
  # Loop over items rated by this user
  for (item,rating) in userRatings.items( ):

    # Loop over items similar to this one
    for (similarity,item2) in itemMatch[item]:

      # Ignore if this user has already rated this item
      if item2 in userRatings: continue
      # Weighted sum of rating times similarity
      scores.setdefault(item2,0)
      scores[item2]+=similarity*rating
      # Sum of all the similarities
      totalSim.setdefault(item2,0)
      totalSim[item2]+=similarity

  # Divide each total score by total weighting to get an average
  rankings=[(score/totalSim[item],item) for item,score in scores.items( )]

  # Return the rankings from highest to lowest
  rankings.sort( )
  rankings.reverse( )
  return rankings

In [57]:
getRecommendedItems(critics, itemsim, 'Toby')

[(3.182634730538922, 'The Night Listener'),
 (2.5983318700614575, 'Just My Luck'),
 (2.4730878186968837, 'Lady in the Water')]

여기서 명심할 점은 getRecommendedItem함수는 항목 유사도 데이터 세트가 미리 준비되어 있어 다른 평론가들의 유사도 점수를 계산하지 않는다.

# 8. 무비렌즈 데이터 세트 이용하기

무비렌즈 다운로드 하기[http://grouplens.org/datasets/movielens/]

u.item, u.data 두 파일만 이용.

<img src=./img/movielens.png width=600>

무비렌즈 데이터를 읽는 코드

In [60]:
def loadMovieLens100k(path='./ml100k'):
  # Get movie titles
  movies={}
  for line in open(path+'/u.item'):
    (id,title)=line.split('|')[0:2]
    movies[id]=title
  
  # Load data
  prefs={}
  for line in open(path+'/u.data'):
    (user,movieid,rating,ts)=line.split('\t')
    prefs.setdefault(user,{})
    prefs[user][movies[movieid]]=float(rating)
  return prefs


In [62]:
prefs = loadMovieLens100k()
prefs['87']

{'2001: A Space Odyssey (1968)': 5.0,
 'Ace Ventura: Pet Detective (1994)': 4.0,
 'Addams Family Values (1993)': 2.0,
 'Addicted to Love (1997)': 4.0,
 'Adventures of Priscilla, Queen of the Desert, The (1994)': 3.0,
 'Adventures of Robin Hood, The (1938)': 5.0,
 'Air Force One (1997)': 3.0,
 'Air Up There, The (1994)': 3.0,
 'Alien (1979)': 4.0,
 'American President, The (1995)': 5.0,
 'Annie Hall (1977)': 4.0,
 'Apocalypse Now (1979)': 4.0,
 'Babe (1995)': 5.0,
 'Baby-Sitters Club, The (1995)': 2.0,
 'Back to the Future (1985)': 5.0,
 'Bad Boys (1995)': 4.0,
 'Bananas (1971)': 5.0,
 'Barcelona (1994)': 3.0,
 'Batman & Robin (1997)': 4.0,
 'Batman (1989)': 3.0,
 'Batman Returns (1992)': 3.0,
 'Big Green, The (1995)': 3.0,
 'Big Squeeze, The (1996)': 2.0,
 'Birdcage, The (1996)': 4.0,
 'Blade Runner (1982)': 4.0,
 'Blues Brothers, The (1980)': 5.0,
 'Boomerang (1992)': 3.0,
 'Boot, Das (1981)': 4.0,
 'Brady Bunch Movie, The (1995)': 2.0,
 'Braveheart (1995)': 4.0,
 'Bridge on the River

데이터를 로드하고, 임의의 사용자 평가 점수를 살펴 본다.(책과는 예제가 다름)

사용자 추천 기법을 적용

In [63]:
getRecommendations(prefs, '87')[0:30]

[(5.0, 'They Made Me a Criminal (1939)'),
 (5.0, 'Star Kid (1997)'),
 (5.0, 'Santa with Muscles (1996)'),
 (5.0, 'Saint of Fort Washington, The (1993)'),
 (5.0, 'Marlene Dietrich: Shadow and Light (1996) '),
 (5.0, 'Great Day in Harlem, A (1994)'),
 (5.0, 'Entertaining Angels: The Dorothy Day Story (1996)'),
 (5.0, 'Boys, Les (1997)'),
 (4.89884443128923, 'Legal Deceit (1997)'),
 (4.815019082242709, 'Letter From Death Row, A (1998)'),
 (4.7321082983941425, 'Hearts and Minds (1996)'),
 (4.696244466490867, 'Pather Panchali (1955)'),
 (4.652397061026758, 'Lamerica (1994)'),
 (4.538723693474813, 'Leading Man, The (1996)'),
 (4.535081339106103, 'Mrs. Dalloway (1997)'),
 (4.532337612572981, 'Innocents, The (1961)'),
 (4.527998574747079, 'Casablanca (1942)'),
 (4.510270149719864, 'Everest (1998)'),
 (4.493967755428439, 'Dangerous Beauty (1998)'),
 (4.485151301801342, 'Wallace & Gromit: The Best of Aardman Animation (1996)'),
 (4.463287461290222, 'Wrong Trousers, The (1993)'),
 (4.450979436941

항목기반 추천 기법 적용<br>
항목기반 데이터셋 작성

In [64]:
itemsim = calculateSimilarItems(prefs, n=50)

100 / 1664
200 / 1664
300 / 1664
400 / 1664
500 / 1664
600 / 1664
700 / 1664
800 / 1664
900 / 1664
1000 / 1664
1100 / 1664
1200 / 1664
1300 / 1664
1400 / 1664
1500 / 1664
1600 / 1664


In [65]:
getRecommendedItems(prefs,itemsim,'87')[0:30]

[(5.0, "What's Eating Gilbert Grape (1993)"),
 (5.0, 'Vertigo (1958)'),
 (5.0, 'Usual Suspects, The (1995)'),
 (5.0, 'Toy Story (1995)'),
 (5.0, 'Titanic (1997)'),
 (5.0, 'Sword in the Stone, The (1963)'),
 (5.0, 'Stand by Me (1986)'),
 (5.0, 'Sling Blade (1996)'),
 (5.0, 'Silence of the Lambs, The (1991)'),
 (5.0, 'Shining, The (1980)'),
 (5.0, 'Shine (1996)'),
 (5.0, 'Sense and Sensibility (1995)'),
 (5.0, 'Scream (1996)'),
 (5.0, 'Rumble in the Bronx (1995)'),
 (5.0, 'Rock, The (1996)'),
 (5.0, 'Robin Hood: Prince of Thieves (1991)'),
 (5.0, 'Reservoir Dogs (1992)'),
 (5.0, 'Police Story 4: Project S (Chao ji ji hua) (1993)'),
 (5.0, 'House of the Spirits, The (1993)'),
 (5.0, 'Fresh (1994)'),
 (5.0, 'Denise Calls Up (1995)'),
 (5.0, 'Day the Sun Turned Cold, The (Tianguo niezi) (1994)'),
 (5.0, 'Before the Rain (Pred dozhdot) (1994)'),
 (5.0, 'Assignment, The (1997)'),
 (5.0, '1-900 (1994)'),
 (4.875, "Ed's Next Move (1996)"),
 (4.833333333333333, 'Anna (1996)'),
 (4.8, 'Dark City 

항목기반 유사도 딕션어리를 만드는데 시간이 많이 걸리지만, 일단 만들고 나면 추천은 거의 즉시 나타난다.<br>
추천 목록을 얻는데 소요되는 시간이 사용자수가 많아져도 증가하지 않는다.

그룹렌즈 웹 사이트에는 책, 농담, 영화 같은 다른 데이터가 더 있으니 실습을 해보세요

#9. 사용자 기반 필터링과 항목 기반 필터링

<img src=./img/algorithms.png width=600>

참고 링크[http://www.kthdaisy.com/recommendation_system_kthdaisy/]

#사용자 기반(user-user)

두 사용자의 공통된 아이템을 기반으로 얼마나 많이 일치하는지 거리를 수치화하는 방법을 사용합니다. 예를 들어, 짐과 제인이 각각 트론이라는 영화에 5점을 입력하였다면, 그 둘 사이의 거리는 0입니다. 반면, 한명은 5 점, 나머지 한명은 3점을 입력하였다면 거리는 증가하게 됩니다. 이러한 사용자간의 유사도를 활용하여 추천하는 방식이 사용자 기반이라고 볼 수 있겠습니다. 그러나, 사용자 기반의 필터링은 공통점을 이끌어내는 것이 쉽지 않다는 문제점이 존재합니다. 사용자 간의 공통점이 추천에 활용할 수 없을 많큼 데이터가 부족할 수 있기 때문입니다. 한가지 예를 들면, 대부분의 사람들은 블록버스터 영화에 선호도가 집중하는 경향이 있습니다. 대부분의 데이터가 블록 버스터 영화에만 데이터가 몰려있기 때문에 추천의 결과가 모든 사용자들에게 동일한 현상이 발생할 확률이 높으며, 이는 추천에 대해 무의미한 결과를 가져오기도 합니다. 또는 웹사이트를 기준으로 볼 때 어떤 페이지의 특정 부분을 클릭하는데 많은 시간이 걸리기 때문에 데이터 수집 자체에 어려움을 겪을 수도 있습니다. 간단한 설명이라 부족한 점이 있지만, 추천에 있어 적절한 데이터 양의 부족으로 인해 공통요소(분류기준)을 분류한다는 게 아주 단순하다고는 볼 수 없습니다. 이를 보완할 수 있는 방법 중 하나가 아이템 기반이라 할 수 있습니다.

#아이템 기반 (item-item)

오늘날 대부분의 추천시스템은 사용자 기반 대신 아이템 간의 거리를 사용하는 아이템 기반을 활용합니다. 넷플릭스와 아마존이 세부적인 내용은 공개하진 않지만, 아이템 기반의 다양한 알고리즘을 사용한다고 공개적으로 언급하고 있습니다. 아이템 기반에 대해 예를 들어 설명을 간략히 하면, 톰클랜시의 책을 좋아하는 사람들이 Clive Cussler의 책도 아주 좋아한다면,Clancy 와 Cussler는 아주 가깝다고 볼 수 있습니다. 즉, 사용자기반과 동일하게 유사도를 기준으로 아이템 간의 거리를 측정한 후 유사한 아이템을 추천하는 방식을 취한다고 볼 수 있습니다. 대부분 사용자대비 아이템이 적은 수를 차지하고 있기 때문에 아이템을 기준으로 볼 때 아주 다양한 관계 데이터가 발생할 확률이 높다고 볼 수 있습니다. 이러한 방식은 데이터가 누적 될수록 추천의 정확도가 높일 수 있는 가능성이 있습니다.

참고로, 원하는 전화번호를 쉽게 찾아 전화 할 수 있는 대표적인 114전국전화 앱에서도 아이템 기반의 분석을 통해, 추천 검색어, 검색품질 개선에 사용되고 있습니다.

Deep Learning Based Collaborative Filtering

<img src=./img/RBMS.png width=600>